In [3]:
import pandas as pd
from apify_client import ApifyClient
from datetime import datetime


apify_client = ApifyClient('apify_api_IY3hoxDRTo2d2iF7NoZXPIOtFlDb8J1cSkPQ')

In [4]:
# Start an actor and wait for it to finish
position_df = pd.DataFrame()
job_titles = ["Data Analyst", "Data Engineer"]#, "Data Scientist", "Software Developer", "Product Manager", "Digital Marketer"]
locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver"} 
# locations = {"w+CAIQICIHVG9yb250bw==": "Toronto", "w+CAIQICIJVmFuY291dmVy": "Vancourver",
#              "w+CAIQICIITW9udHJlYWw=": "Montreal", "w+CAIQICIHQ2FsZ2FyeQ==": "Calgary", "w+CAIQICIIRWRtb250b24=": "Edmonton"}  # UULE

for uule in locations:
    print(locations[uule])
    for jt in job_titles:
        print(jt)
        run_input = {
            "csvFriendlyOutput": True,
            "includeUnfilteredResults": False,
            "maxConcurrency": 10,
            "maxPagesPerQuery": 3,
            "queries": f"https://www.google.com/search?ibp=htl;jobs&q={jt}&uule={uule}",
            "saveHtml": False,
            "saveHtmlToKeyValueStore": False,
        }

        actor_call = apify_client.actor(
            'dan.scraper/google-jobs-scraper').call(run_input=run_input)

        dataset_items = apify_client.dataset(
            actor_call['defaultDatasetId']).list_items().items

        d = pd.DataFrame(dataset_items)
        d["query"] = jt
        d["location"] = locations[uule]
        d["run_time"] = str(datetime.now())

        position_df = pd.concat([position_df, d])

    print("="*30)
    break # Only Run For Toronto

Toronto
Data Analyst
Data Engineer


In [5]:
position_df.groupby("location")["query"].value_counts()

location  query        
Toronto   Data Analyst     20
          Data Engineer    20
Name: count, dtype: int64

In [18]:
#_ = position_df.pop("thumbnail") # not useful, with na values
_ = position_df.insert(1,"publishDate", pd.Timestamp.now())
#_ = position_df.pop("publishDate")
position_df.to_csv("../data/raw_google_1129.csv", index=False)


In [19]:
position_df.head()

,title,publishDate,companyName,location,via,description,jobHighlights,relatedLinks,extras,metadata,applyLink,query,run_time
0,"Data Analyst, Product Analytics",2024-03-03 21:23:37.296032,Thomson Reuters,Toronto,via Thomson Reuters Careers,"Data Analyst, Product Analytics\n\nIf you’re a...","[{'items': ['Data Analyst, Product Analytics ...","[{'link': 'http://www.tr.com/', 'text': 'tr.co...","[4 days ago, Full-time, No degree mentioned]","{'postedAt': '4 days ago', 'scheduleType': 'Fu...",{'title': '.nFg2eb{font-weight:500}.Bi6Ddc{fon...,Data Analyst,2024-03-03 21:00:13.380198
1,Data Analyst,2024-03-03 21:23:37.296032,Resrch Lab,Toronto,via LinkedIn,"Description: As a Data Analyst, you'll be resp...","[{'items': ['Description: As a Data Analyst, y...",[{'link': 'https://www.google.com/search?sca_e...,"[1 day ago, Part-time, No degree mentioned]","{'postedAt': '1 day ago', 'scheduleType': 'Par...","{'title': 'Apply on LinkedIn', 'link': 'https:...",Data Analyst,2024-03-03 21:00:13.380198
2,Data Analyst,2024-03-03 21:23:37.296032,Equinix,Toronto,via Career - Equinix,Who are we?\n\nEquinix is the world’s digital ...,[{'items': ['Who are we? Equinix is the world...,"[{'link': 'http://www.equinix.com/', 'text': '...","[4 days ago, Full-time]","{'postedAt': '4 days ago', 'scheduleType': 'Fu...","{'title': 'Apply on Career - Equinix', 'link':...",Data Analyst,2024-03-03 21:00:13.380198
3,Senior Data Analyst,2024-03-03 21:23:37.296032,Royal Bank of Canada,Toronto,via RBC Careers,Job Summary\n\nJob Description\n\nWhat is the ...,[{'items': ['Job Summary Job Description Wha...,"[{'link': 'http://www.rbc.com/', 'text': 'rbc....","[4 days ago, Full-time]","{'postedAt': '4 days ago', 'scheduleType': 'Fu...","{'title': 'Apply on RBC Careers', 'link': 'htt...",Data Analyst,2024-03-03 21:00:13.380198
4,HR Systems and Data Analyst - 15 Month Contract,2024-03-03 21:23:37.296032,Meridian Credit Union,Toronto,via ZipRecruiter,About this opportunity\n\nAt Meridian our aspi...,[{'items': ['About this opportunity At Meridi...,"[{'link': 'http://www.meridiancu.ca/', 'text':...","[1 day ago, Work from home, Full-time and Cont...","{'postedAt': '1 day ago', 'scheduleType': 'Ful...","{'title': 'Apply on ZipRecruiter', 'link': 'ht...",Data Analyst,2024-03-03 21:00:13.380198


In [20]:
from opensearchpy import OpenSearch, helpers

In [8]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin') # For testing only. Don't store credentials in code.
ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    ca_certs = ca_certs_path
)

print(client.info)

<bound method OpenSearch.info of <OpenSearch([{'host': 'localhost', 'port': 9200}])>>


In [10]:
index_name = "swift_dev"

if not client.indices.exists(index_name):
    client.indices.create(index=index_name)


In [12]:
def doc_generator(df):
    for i, row in df.iterrows():
        doc = {
            "_index": index_name,
            "_source": row.to_dict(),
        }
        yield doc

helpers.bulk(client, doc_generator(position_df))

print("Data Saved to ES")

Data Saved to ES


In [13]:
position_df["query"].value_counts()

query
Data Analyst     30
Data Engineer    30
Name: count, dtype: int64